In [1]:
pip install keras==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.3 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install patchify

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from math import log2
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model

def mlp_block(inputs, config):
    x = layers.Dense(config["mlp_units"], activation="gelu")(inputs)
    x = layers.Dropout(config["dropout_rate"])(x)
    x = layers.Dense(config["hidden_units"])(x)
    x = layers.Dropout(config["dropout_rate"])(x)
    return x

def transformer_block(inputs, config):
    skip_conn1 = inputs
    x = layers.LayerNormalization()(inputs)
    x = layers.MultiHeadAttention(
        num_heads=config["heads"], key_dim=config["hidden_units"]
    )(x, x)
    x = layers.Add()([x, skip_conn1])

    skip_conn2 = x
    x = layers.LayerNormalization()(x)
    x = mlp_block(x, config)
    x = layers.Add()([x, skip_conn2])

    return x

def conv_layer_block(inputs, filters, kernel_size=3):
    x = layers.Conv2D(filters, kernel_size=kernel_size, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def upconv_layer_block(inputs, filters, strides=2):
    x = layers.Conv2DTranspose(filters, kernel_size=2, padding="same", strides=strides)(inputs)
    return x

def build_custom_unet(config):
  
    input_shape = (config["num_patches"], config["patch_size"] * config["patch_size"] * config["channels"])
    inputs = layers.Input(input_shape)

        
    patch_embedding = layers.Dense(config["hidden_units"])(inputs)  # (None, 256, 768)

    positions = tf.range(start=0, limit=config["num_patches"], delta=1)  # (256,)
    pos_embedding = layers.Embedding(input_dim=config["num_patches"], output_dim=config["hidden_units"])(positions)  # (256, 768)
    x = patch_embedding + pos_embedding  # (None, 256, 768)

    #Transformer Encoder
    skip_conn_indices = [3, 6, 9, 12]
    skip_connections = []

    for i in range(1, config["layers"] + 1):
        x = transformer_block(x, config)

        if i in skip_conn_indices:
            skip_connections.append(x)

   
    z3, z6, z9, z12 = skip_connections

  
    z0 = layers.Reshape((config["img_size"], config["img_size"], config["channels"]))(inputs)

    shape = (
        config["img_size"] // config["patch_size"],
        config["img_size"] // config["patch_size"],
        config["hidden_units"]
    )
    z3 = layers.Reshape(shape)(z3)
    z6 = layers.Reshape(shape)(z6)
    z9 = layers.Reshape(shape)(z9)
    z12 = layers.Reshape(shape)(z12)

   
    total_upscale_factor = int(log2(config["patch_size"]))
    upscale = total_upscale_factor - 4

    if upscale >= 2:  # Patch size 16 or greater
        z3 = upconv_layer_block(z3, z3.shape[-1], strides=2 ** upscale)
        z6 = upconv_layer_block(z6, z6.shape[-1], strides=2 ** upscale)
        z9 = upconv_layer_block(z9, z9.shape[-1], strides=2 ** upscale)
        z12 = upconv_layer_block(z12, z12.shape[-1], strides=2 ** upscale)

    if upscale < 0:  # Patch size less than 16
        p = 2 ** abs(upscale)
        z3 = layers.MaxPool2D((p, p))(z3)
        z6 = layers.MaxPool2D((p, p))(z6)
        z9 = layers.MaxPool2D((p, p))(z9)
        z12 = layers.MaxPool2D((p, p))(z12)

    #Decoder 1
    x = upconv_layer_block(z12, 128)

    s = upconv_layer_block(z9, 128)
    s = conv_layer_block(s, 128)

    x = layers.Concatenate()([x, s])
    x = conv_layer_block(x, 128)
    x = conv_layer_block(x, 128)

    # Decoder 2 
    x = upconv_layer_block(x, 64)

    s = upconv_layer_block(z6, 64)
    s = conv_layer_block(s, 64)
    s = upconv_layer_block(s, 64)
    s = conv_layer_block(s, 64)

    x = layers.Concatenate()([x, s])
    x = conv_layer_block(x, 64)
    x = conv_layer_block(x, 64)

    #Decoder 3
    x = upconv_layer_block(x, 32)

    s = upconv_layer_block(z3, 32)
    s = conv_layer_block(s, 32)
    s = upconv_layer_block(s, 32)
    s = conv_layer_block(s, 32)
    s = upconv_layer_block(s, 32)
    s = conv_layer_block(s, 32)

    x = layers.Concatenate()([x, s])
    x = conv_layer_block(x, 32)
    x = conv_layer_block(x, 32)

    #Decoder 4 
    x = upconv_layer_block(x, 16)

    s = conv_layer_block(z0, 16)
    s = conv_layer_block(s, 16)

    x = layers.Concatenate()([x, s])
    x = conv_layer_block(x, 16)
    x = conv_layer_block(x, 16)

    #output
    outputs = layers.Conv2D(1, kernel_size=1, padding="same", activation="sigmoid")(x)

    return Model(inputs, outputs, name="Custom_UNETR_2D")

if __name__ == "__main__":
    config = {}
    config["img_size"] = 512
    config["layers"] = 12
    config["hidden_units"] = 64
    config["mlp_units"] = 128
    config["heads"] = 6
    config["dropout_rate"] = 0.1
    config["patch_size"] = 512
    config["num_patches"] = (config["img_size"] ** 2) // (config["patch_size"] ** 2)
    config["channels"] = 3

    model = build_custom_unet(config)
    model.summary()


2024-07-25 05:37:33.380352: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 05:37:33.380420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 05:37:33.382137: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model: "Custom_UNETR_2D"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1, 786432)]          0         []                            
                                                                                                  
 dense (Dense)               (None, 1, 64)                5033171   ['input_1[0][0]']             
                                                          2                                       
                                                                                                  
 tf.__operators__.add (TFOp  (None, 1, 64)                0         ['dense[0][0]']               
 Lambda)                                                                                          
                                                                                    

In [7]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
import numpy as np
import tensorflow as tf

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

2024-07-25 07:53:31.456347: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 07:53:31.456482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 07:53:31.609289: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from math import log2
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model

def mlp_block(inputs, config):
    x = layers.Dense(config["mlp_units"], activation="gelu")(inputs)
    x = layers.Dropout(config["dropout_rate"])(x)
    x = layers.Dense(config["hidden_units"])(x)
    x = layers.Dropout(config["dropout_rate"])(x)
    return x

def transformer_block(inputs, config):
    skip_conn1 = inputs
    x = layers.LayerNormalization()(inputs)
    x = layers.MultiHeadAttention(
        num_heads=config["heads"], key_dim=config["hidden_units"]
    )(x, x)
    x = layers.Add()([x, skip_conn1])

    skip_conn2 = x
    x = layers.LayerNormalization()(x)
    x = mlp_block(x, config)
    x = layers.Add()([x, skip_conn2])

    return x

def conv_layer_block(inputs, filters, kernel_size=3):
    x = layers.Conv2D(filters, kernel_size=kernel_size, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    return x

def upconv_layer_block(inputs, filters, strides=2):
    x = layers.Conv2DTranspose(filters, kernel_size=2, padding="same", strides=strides)(inputs)
    return x

def build_custom_unet(config):
  
    input_shape = (config["num_patches"], config["patch_size"] * config["patch_size"] * config["channels"])
    inputs = layers.Input(input_shape)

        
    patch_embedding = layers.Dense(config["hidden_units"])(inputs)  # (None, 256, 768)

    positions = tf.range(start=0, limit=config["num_patches"], delta=1)  # (256,)
    pos_embedding = layers.Embedding(input_dim=config["num_patches"], output_dim=config["hidden_units"])(positions)  # (256, 768)
    x = patch_embedding + pos_embedding  # (None, 256, 768)

    #Transformer Encoder
    skip_conn_indices = [3, 6, 9, 12]
    skip_connections = []

    for i in range(1, config["layers"] + 1):
        x = transformer_block(x, config)

        if i in skip_conn_indices:
            skip_connections.append(x)

   
    z3, z6, z9, z12 = skip_connections

  
    z0 = layers.Reshape((config["img_size"], config["img_size"], config["channels"]))(inputs)

    shape = (
        config["img_size"] // config["patch_size"],
        config["img_size"] // config["patch_size"],
        config["hidden_units"]
    )
    z3 = layers.Reshape(shape)(z3)
    z6 = layers.Reshape(shape)(z6)
    z9 = layers.Reshape(shape)(z9)
    z12 = layers.Reshape(shape)(z12)

   
    total_upscale_factor = int(log2(config["patch_size"]))
    upscale = total_upscale_factor - 4

    if upscale >= 2:  # Patch size 16 or greater
        z3 = upconv_layer_block(z3, z3.shape[-1], strides=2 ** upscale)
        z6 = upconv_layer_block(z6, z6.shape[-1], strides=2 ** upscale)
        z9 = upconv_layer_block(z9, z9.shape[-1], strides=2 ** upscale)
        z12 = upconv_layer_block(z12, z12.shape[-1], strides=2 ** upscale)

    if upscale < 0:  # Patch size less than 16
        p = 2 ** abs(upscale)
        z3 = layers.MaxPool2D((p, p))(z3)
        z6 = layers.MaxPool2D((p, p))(z6)
        z9 = layers.MaxPool2D((p, p))(z9)
        z12 = layers.MaxPool2D((p, p))(z12)

    #Decoder 1
    x = upconv_layer_block(z12, 128)

    s = upconv_layer_block(z9, 128)
    s = conv_layer_block(s, 128)

    x = layers.Concatenate()([x, s])
    x = conv_layer_block(x, 128)
    x = conv_layer_block(x, 128)

    # Decoder 2 
    x = upconv_layer_block(x, 64)

    s = upconv_layer_block(z6, 64)
    s = conv_layer_block(s, 64)
    s = upconv_layer_block(s, 64)
    s = conv_layer_block(s, 64)

    x = layers.Concatenate()([x, s])
    x = conv_layer_block(x, 64)
    x = conv_layer_block(x, 64)

    #Decoder 3
    x = upconv_layer_block(x, 32)

    s = upconv_layer_block(z3, 32)
    s = conv_layer_block(s, 32)
    s = upconv_layer_block(s, 32)
    s = conv_layer_block(s, 32)
    s = upconv_layer_block(s, 32)
    s = conv_layer_block(s, 32)

    x = layers.Concatenate()([x, s])
    x = conv_layer_block(x, 32)
    x = conv_layer_block(x, 32)

    #Decoder 4 
    x = upconv_layer_block(x, 16)

    s = conv_layer_block(z0, 16)
    s = conv_layer_block(s, 16)

    x = layers.Concatenate()([x, s])
    x = conv_layer_block(x, 16)
    x = conv_layer_block(x, 16)

    #output
    outputs = layers.Conv2D(1, kernel_size=1, padding="same", activation="sigmoid")(x)

    return Model(inputs, outputs, name="Custom_UNETR_2D")

if __name__ == "__main__":
    config = {}
    config["img_size"] = 512
    config["layers"] = 12
    config["hidden_units"] = 64
    config["mlp_units"] = 128
    config["heads"] = 6
    config["dropout_rate"] = 0.1
    config["patch_size"] = 512
    config["num_patches"] = (config["img_size"] ** 2) // (config["patch_size"] ** 2)
    config["channels"] = 3

    model = build_custom_unet(config)
    model.summary()


Model: "Custom_UNETR_2D"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1, 786432)]          0         []                            
                                                                                                  
 dense (Dense)               (None, 1, 64)                5033171   ['input_1[0][0]']             
                                                          2                                       
                                                                                                  
 tf.__operators__.add (TFOp  (None, 1, 64)                0         ['dense[0][0]']               
 Lambda)                                                                                          
                                                                                    

In [7]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from patchify import patchify

#configuration
cf = {}
cf["img_size"] = 256
cf["channels"] = 3
cf["layers"] = 12
cf["hidden_units"] = 128
cf["mlp_units"] = 32
cf["heads"] = 6
cf["dropout_rate"] = 0.1
cf["patch_size"] = 16
cf["num_patches"] = (cf["img_size"]**2)//(cf["patch_size"]**2)
cf["flat_patches_shape"] = (
    cf["num_patches"],
    cf["patch_size"]*cf["patch_size"]*cf["channels"]
)

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split=0.1):

    X = sorted(glob(os.path.join(path, "train", "*_sat.jpg")))
    Y = sorted(glob(os.path.join(path, "train", "*_mask.png")))

    split_size = int(len(X) * split)

    train_x, valid_x = train_test_split(X, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(Y, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (cf["img_size"], cf["img_size"]))
    image = image / 255.0


    patch_shape = (cf["patch_size"], cf["patch_size"], cf["channels"])
    patches = patchify(image, patch_shape, cf["patch_size"])
    patches = np.reshape(patches, cf["flat_patches_shape"])
    patches = patches.astype(np.float32)

    return patches

def read_mask(path):
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (cf["img_size"], cf["img_size"]))
    mask = mask / 255.0
    mask = mask.astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape(cf["flat_patches_shape"])
    y.set_shape([cf["img_size"], cf["img_size"], 1])
    return x, y

def tf_dataset(X, Y, batch=2):
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    ds = ds.map(tf_parse).batch(batch).prefetch(10)
    return ds

class SaveModelEveryN(Callback):
    def __init__(self, save_path, n_epochs):
        super(SaveModelEveryN, self).__init__()
        self.save_path = save_path
        self.n_epochs = n_epochs

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.n_epochs == 0:
            save_path = os.path.join(self.save_path, f"model_epoch_{epoch+1}.h5")
            self.model.save(save_path)
            print(f"\nModel saved to {save_path}")

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)
    
    create_dir("files")

    # Hyperparameters
    batch_size = 8
    lr = 0.1
    num_epochs = 500
    model_path = os.path.join("files", "model_best.h5")
    csv_path = os.path.join("files", "log.csv")
    save_dir = "files"

    dataset_path = "/kaggle/input/deepglobe-road-extraction-dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)


    model = build_custom_unet(cf)
    #model.load_weights("/kaggle/input/road-extraction-transformer-based-unet/keras/40-epoch/1/model_epoch_40.h5")
    model.compile(loss=dice_loss, optimizer=SGD(lr), metrics=['accuracy'])
    # model.summary()

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor='val_loss', mode='min'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
        SaveModelEveryN(save_path=save_dir, n_epochs=10)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Epoch 1/500


KeyboardInterrupt: 

In [17]:
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt

cf = {}
cf["img_size"] = 256
cf["channels"] = 3

if __name__ == "__main__":
   
    np.random.seed(42)
    tf.random.set_seed(42)
   
    def create_dir(path):
        if not os.path.exists(path):
            os.makedirs(path)

    create_dir(f"results_UNET_TR_v3")

    model_path = r"/kaggle/input/custom-transformer-based-unet/keras/default/1/UNET_Transformer_final.h5"
    model = tf.keras.models.load_model(model_path, custom_objects={"dice_loss": dice_loss})

    dataset_path = r"/kaggle/input/deepglobe-road-extraction-dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)

    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        name = x.split("/")[-1]
        
        img = plt.imread(x)
        mask=plt.imread(y)
        img = cv2.resize(img, (256, 256))
        mask = cv2.resize(mask, (256, 256))
        
        if img.shape[-1] == 4:  
            img = img[..., :3]  #remving the alpha chanel
        elif img.shape[-1] == 1:  # If image is grayscale
            img = np.repeat(img, 3, axis=-1)  # Converting to rgb

        # Ensure the mask has 1 channel
        if mask.ndim == 3 and mask.shape[-1] == 3:
            mask = mask[..., 0]  
        img = np.expand_dims(img, axis=0)  
        mask = np.expand_dims(mask, axis=-1) 

        print(f"Image shape: {img.shape}, Mask shape: {mask.shape}")
  
        frameObj={}
        frameObj['img']=img
        frameObj['mask']=mask[:,:,0]
       
        img = frameObj['img']
        mask = frameObj['mask']
        imgProc = img
        
    
        plt.figure(figsize=(10, 10))
    
        plt.subplot(1, 3, 1)
        plt.imshow(img[0])
        plt.title('Aerial image')
        
        plt.subplot(1, 3, 3)
        plt.imshow(mask[:, :, 0] if mask.ndim == 3 else mask, cmap='gray')
        plt.title('Actual Routes')

        predictions = model.predict(img)
        predictions = (predictions > 0.1).astype(np.float32)  # Thresholding to get binary mask
        plt.subplot(1, 3, 2)
        plt.imshow(predictions[0, :, :, 0], cmap='gray')
        plt.title('Predicted Routes')
#         plt.show()
        
        plt.savefig(f'results_UNET_TR_v3/{name}.png')
        plt.close() 

      
 


  0%|          | 0/622 [00:00<?, ?it/s]

Image shape: (1, 256, 256, 3), Mask shape: (256, 256, 1)
1/1 [==============================] - 1s 608ms/step


  0%|          | 1/622 [00:01<12:00,  1.16s/it]

Image shape: (1, 256, 256, 3), Mask shape: (256, 256, 1)
1/1 [==============================] - 0s 113ms/step


  0%|          | 2/622 [00:01<08:40,  1.19it/s]

...

100%|█████████▉| 620/622 [06:57<00:01,  1.52it/s]

Image shape: (1, 256, 256, 3), Mask shape: (256, 256, 1)
1/1 [==============================] - 0s 101ms/step


100%|█████████▉| 621/622 [06:58<00:00,  1.54it/s]

Image shape: (1, 256, 256, 3), Mask shape: (256, 256, 1)
1/1 [==============================] - 0s 102ms/step


100%|██████████| 622/622 [06:59<00:00,  1.48it/s]


In [18]:
import shutil


folder_to_zip = "results_UNET_TR_v3"
zip_filename = "results_UNET_TR_v3.zip"


shutil.make_archive(folder_to_zip, 'zip', folder_to_zip)

from IPython.display import FileLink
FileLink(zip_filename)

/kaggle/working/results_UNET_TR_v3.zip